In [1]:
import pandas as pd
import numpy as np
import datetime
import iexfinance
from iexfinance.stocks import get_historical_intraday
from sklearn.externals import joblib

# Testdata_download_from_iex

In [2]:
def get_sp500(k=3):
    sp500=pd.read_excel('C:/Users/user/Documents/Модуль биржа по фракталам/sp500_symbols.xlsx')
    sp500=list(sp500['symbol'])
    return sp500[1:len(sp500):k]

def testdata_download_from_iex(start_day, end_day):
    dti=pd.date_range(start_day,end_day, freq='B')
    sp500=get_sp500(k=3)
    for date in dti:
        print(date)
        companies={}
        for i in sp500:
            try:
                companies[i]=get_historical_intraday(symbol=i, date=date, output_format='pandas', token="sk_1dfa8856748344d293525acdfbfb0835")
            except:
                pass

        for i in companies.keys():
            try:
                companies[i].to_excel('C:/Users/user/Documents/Модуль биржа по фракталам/iex_intraday/'+str(i)+'_'+str(date)[:-9]+'.xlsx')
            except:
                pass

In [ ]:
testdata_download_from_iex(start_day=datetime.datetime(2019,7,3),
                           end_day=datetime.datetime(2019,7,10))


2019-07-03 00:00:00
2019-07-04 00:00:00
2019-07-05 00:00:00
2019-07-08 00:00:00
2019-07-09 00:00:00
2019-07-10 00:00:00
2019-07-11 00:00:00


# Recommends

In [2]:
ready_dataset_for_marking = pd.read_excel('ready_dataset_for_marking_last.xlsx')

In [3]:
scaler = joblib.load('scaler_best.pkl')
X_real_test = scaler.transform(ready_dataset_for_marking)

xgb_clf_from_disk = joblib.load('xgb_clf_best.pkl')
forest_clf_from_disk = joblib.load('forest_clf_best.pkl')
gboost_clf_from_disk = joblib.load('gboost_clf_best.pkl')
bernoulli_clf_from_disk = joblib.load('bernoulli_clf_best.pkl')
xgb_clf_for_df_predictions_from_disk = joblib.load('xgb_clf_for_df_predictions_best.pkl')

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [4]:
models = [xgb_clf_from_disk, forest_clf_from_disk, gboost_clf_from_disk, bernoulli_clf_from_disk]
dict_predict = {}
for model in models:
    y_test_pred = model.predict_proba(X_real_test)
    dict_predict[str(model)[:9]+' proba_0'] = np.array([i[0] for i in y_test_pred])
    dict_predict[str(model)[:9]+' proba_1'] = np.array([i[1] for i in y_test_pred])
    dict_predict[str(model)[:9]+' proba_2'] = np.array([i[2] for i in y_test_pred])
df_predictions = pd.DataFrame(dict_predict, index=ready_dataset_for_marking.index)

In [6]:
y_test_pred = xgb_clf_for_df_predictions_from_disk.predict(df_predictions)
y_test_pred_proba = xgb_clf_for_df_predictions_from_disk.predict_proba(df_predictions)

In [7]:
df=pd.DataFrame({'predictions':y_test_pred,'predictions_proba_for_2':[i[2] for i in y_test_pred_proba]}, index=ready_dataset_for_marking.index)
df[df.predictions==2]

,predictions,predictions_proba_for_2
2019-03-29 15:00:00,2,0.533830
2019-04-04 12:00:00,2,0.641920
2019-04-04 15:00:00,2,0.566012
2019-04-05 12:00:00,2,0.681404
2019-04-05 13:00:00,2,0.593011
2019-04-23 11:00:00,2,0.482617
2019-05-16 14:00:00,2,0.558821
2019-05-21 10:00:00,2,0.655312
2019-06-13 11:00:00,2,0.491199
2019-06-13 14:00:00,2,0.727209


# Одна бумага сыграла роль в погрешности по точности.В общем, надо работать с 500-а бумагами. И скорее всего, персонально
